# CSCI 3352 Project

**Names:** Tom Stoughton, Dylan Smith

Summary: ...

In [2]:
import networkx as nx
import cobra
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
# set up networks here

In [4]:
# numerical optimization code

In [ ]:
# clustering analysis